# Sources

Upstox - https://upstox.com/developer/api-documentation/instruments/
NSE200 - https://www.nseindia.com/products-services/indices-nifty200-index

Vlookup - "C:\Users\perry\OneDrive\nifty200\ind_nifty200list.xlsx"

In [8]:
from datetime import date, timedelta, datetime
import requests

sym_returns = dict()

def datesort(crow):
    dt = crow[0]
    return int(datetime.fromisoformat(dt).timestamp())

def get_12m_returns(isin):
    ed = str(date.today())
    sd = str(date.today() - timedelta(weeks=52))
    url = f"https://api-v2.upstox.com/historical-candle/NSE_EQ%7C{isin}/month/{ed}/{sd}"
    h = {
        "Api-Version": "2.0",
        "Accept": "application/json",
        "Authorization": "***REMOVED***",
    }

    # https://upstox.com/developer/api-documentation/get-historical-candle-data/
    resp = requests.get(url, headers=h)
    rjson = resp.json()
    
    if rjson["status"] != "success":
        raise Exception(rjson["status"])
    
    candles = rjson["data"]["candles"]
    candles.sort(key=datesort, reverse=True)
    for c in candles:
        print(f"date: {c[0]} close: {c[4]}")

    start_price = candles[-1][4]
    end_price = candles[0][4]
    return (end_price - start_price)/start_price

get_12m_returns("INE758T01015")

date: 2024-08-01T00:00:00+05:30 close: 262.65
date: 2024-07-01T00:00:00+05:30 close: 229.45
date: 2024-06-01T00:00:00+05:30 close: 200.56
date: 2024-05-01T00:00:00+05:30 close: 179.15
date: 2024-04-01T00:00:00+05:30 close: 193.15
date: 2024-03-01T00:00:00+05:30 close: 182.1
date: 2024-02-01T00:00:00+05:30 close: 165.45
date: 2024-01-01T00:00:00+05:30 close: 139.55
date: 2023-12-01T00:00:00+05:30 close: 123.7
date: 2023-11-01T00:00:00+05:30 close: 118.55
date: 2023-10-01T00:00:00+05:30 close: 105.1
date: 2023-09-01T00:00:00+05:30 close: 101.5


1.5876847290640392

In [ ]:
# Sort C200 by previous 12 months returns decreasing

from datetime import date, timedelta, datetime
import requests
import pandas as pd

sym_returns = dict()

def datesort(crow):
    dt = crow[0]
    return int(datetime.fromisoformat(dt).timestamp())

def get_12m_returns(isin):
    ed = str(date.today())
    sd = str(date.today() - timedelta(weeks=52))
    url = f"https://api-v2.upstox.com/historical-candle/NSE_EQ%7C{isin}/month/{ed}/{sd}"
    h = {
        "Api-Version": "2.0",
        "Accept": "application/json",
        "Authorization": "***REMOVED***",
    }

    # https://upstox.com/developer/api-documentation/get-historical-candle-data/
    resp = requests.get(url, headers=h)
    if resp.status_code != 200:
        print(f"isin: {isin} err: {resp}")
        return None
    rjson = resp.json()
    
    if rjson["status"] != "success":
        raise Exception(rjson["status"])
    
    candles = rjson["data"]["candles"]
    candles.sort(key=datesort, reverse=True)
    # for c in candles:
    #     print(f"date: {c[0]} close: {c[4]}")
    
    start_price = candles[-1][4]
    end_price = candles[0][4]
    return (end_price - start_price)/start_price

def apply_12m_returns(row):
    sym_returns[row['Symbol']] = get_12m_returns(isin=row['ISIN Code'])

df = pd.read_csv(C200)
_ = df.apply(apply_12m_returns, axis='columns')

t200 = []
for k,v in sorted(sym_returns.items(), key=lambda x: x[1], reverse=True):
    t200.append({"symbol": k, "gain": v})

# print(t200)


In [8]:
# select top 40 (T40) and top 20 (T20) from it.

t40 = t200[:40]
t20 = t200[:20]

[{'symbol': 'OIL', 'gain': 2.4215809284818066}, {'symbol': 'RVNL', 'gain': 2.379758040719977}, {'symbol': 'TRENT', 'gain': 2.3405042905559696}, {'symbol': 'SUZLON', 'gain': 2.0558139534883724}, {'symbol': 'OFSS', 'gain': 1.82405673172559}, {'symbol': 'PRESTIGE', 'gain': 1.7840192499170264}, {'symbol': 'KALYANKJIL', 'gain': 1.5880026367831244}, {'symbol': 'ZOMATO', 'gain': 1.5876847290640392}, {'symbol': 'BDL', 'gain': 1.5516338601764763}, {'symbol': 'HINDPETRO', 'gain': 1.5351672397624259}, {'symbol': 'DIXON', 'gain': 1.5089662995698823}, {'symbol': 'HAL', 'gain': 1.5012317506418067}, {'symbol': 'IRFC', 'gain': 1.3959503592423252}, {'symbol': 'INDUSTOWER', 'gain': 1.265779864371414}, {'symbol': 'TORNTPOWER', 'gain': 1.2640487474610698}, {'symbol': 'BHEL', 'gain': 1.2610687022900762}, {'symbol': 'CUMMINSIND', 'gain': 1.2531097093674466}, {'symbol': 'VEDL', 'gain': 1.2403390675641983}, {'symbol': 'BEL', 'gain': 1.212581344902386}, {'symbol': 'POLICYBZR', 'gain': 1.2031362299901993}, {'sy

In [ ]:
# Open portfolio.csv, which contains all symbols held currently, call it HXX

import pandas as pd

df = pd.read_csv("portfolio.csv")